In [1]:
import pickle

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
cin =16
cout = 33
k_size = 3
stride = 2
pad = 0

b_size = 1
cin = 16
lin = 50

m = nn.Conv1d(cin, cout, kernel_size=k_size, stride=2, padding=0)
input = torch.randn(b_size, cin, lin)
output = m(input)

In [3]:
def len_out(lin, pad, k_size, stride):
    return (lin + 2*pad -k_size)//stride +1
def len_in(lout, pad, k_size, stride):
    return (lout -1)*stride -2*pad + k_size
k_size = 3
stride = 2
pad = 0

lout = 1
for _ in range(3):
    lout = len_in(lout, pad, k_size, stride)
    print(lout)

3
7
15


In [4]:
k_size = 3
stride = 2
pad = 0

lout = 1
list_out =[]
while lout<100:
    lout = len_in(lout, pad, k_size, stride)
    list_out.append(lout)
list_out

[3, 7, 15, 31, 63, 127]

In [5]:
class BaseModule(nn.Module):
    
    def __init__(self, cin, hidden, k_size=3, stride=2, pad=0):
        super(BaseModule, self).__init__()
        self.conv1 = nn.Sequential(
                    nn.Conv1d(cin, hidden, k_size, stride, pad),
                    nn.BatchNorm1d(hidden),
                    nn.ReLU()
                )
        self.linear = nn.Linear(hidden, 1)

    def forward(self, x):
        torch.concat([self.conv1d(i) for i in x])
        x = [self.conv1(i) for i in x]
        x = torch.concat([i for i in range(len(x))], axis=1)
        x = F.adaptive_avg_pool1d(x, (1))
        x = self.linear(x)

        return x

In [6]:
class TestModule(nn.Module):
    
    def __init__(self, cin, hidden, b_size):
        self.k_size = 3
        self.stride = 2
        self.pad = 0
        self.b_size = b_size
        super(TestModule, self).__init__()
        self.conv1 = nn.Sequential(
                    nn.Conv1d(cin, hidden, self.k_size, self.stride, self.pad),
                    nn.Tanh()
                )
        self.linear = nn.Linear(hidden, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = F.adaptive_avg_pool1d(x, (1)).reshape(self.b_size, -1)
        x = self.linear(x)
        return x

## Featuring for embedding model

In [11]:
def labeled_protein(df_protein_sequence, strings):
    labels = {key: str(label) for label,key in enumerate(strings)}
    labeled_protein = []
    length = []
    for protein in df_protein_sequence:
        trg = []
        for key in protein:
            trg.append(int(labels[key]))
        labeled_protein.append(torch.LongTensor(trg))
        length.append(len(trg))

    return labeled_protein, length

In [12]:
from mymodule import except_outlier

with open('dataset/featured/base.pkl', 'rb') as f:
    data = pickle.load(f)
data = except_outlier(data, 'sequence_len')

strings = strings = 'ABCDEFGHIJKLMNOPQRSTUVWSYZ'
proteins_sequence = data['protein_sequence']
labeled_protein, length = labeled_protein(data['protein_sequence'], strings)
data = data[['seq_id', 'protein_sequence', 'pH','tm']]
data['labeled_protein'] = labeled_protein

In [13]:
max_len = max(length)
proteins =  data['labeled_protein']
for idx, protein in enumerate(proteins):
    proteins[idx] = em(protein)
proteins

/var/folders/v7/1tdz9h091g59gfrm4hj34cjc0000gn/T/ipykernel_20435/1028765440.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  proteins[idx] = em(protein)


0        [[tensor(-0.3866, grad_fn=<UnbindBackward0>), ...
1        [[tensor(-0.3866, grad_fn=<UnbindBackward0>), ...
2        [[tensor(-0.3866, grad_fn=<UnbindBackward0>), ...
3        [[tensor(-0.3866, grad_fn=<UnbindBackward0>), ...
4        [[tensor(-0.3866, grad_fn=<UnbindBackward0>), ...
                               ...                        
29368    [[tensor(-0.5095, grad_fn=<UnbindBackward0>), ...
29369    [[tensor(-0.5095, grad_fn=<UnbindBackward0>), ...
29370    [[tensor(-0.5095, grad_fn=<UnbindBackward0>), ...
29371    [[tensor(-0.5095, grad_fn=<UnbindBackward0>), ...
29372    [[tensor(-0.5095, grad_fn=<UnbindBackward0>), ...
Name: labeled_protein, Length: 29373, dtype: object

In [23]:
from mymodule import except_outlier

with open('dataset/featured/base.pkl', 'rb') as f:
    data = pickle.load(f)
data = except_outlier(data, 'sequence_len')

strings = strings = 'ABCDEFGHIJKLMNOPQRSTUVWSYZ'
proteins_sequence = data['protein_sequence']
labeled_protein, length = labeled_protein(data['protein_sequence'], strings)
data = data[['seq_id', 'protein_sequence', 'pH','tm']]
data['labeled_protein'] = labeled_protein






em = nn.Embedding(len(strings), 5)

max_len = max(length)
proteins =  data['labeled_protein']
for idx, protein in enumerate(proteins):
    protein = em(protein)
    proteins[idx] = torch.concat((protein, ))
proteins

TypeError: 'list' object is not callable

In [22]:
em = nn.Embedding()
proteins[0].shape

torch.Size([341, 10])

In [21]:
torch.concat((torch.ones(2,3), torch.zeros(2,3)))

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [0., 0., 0.],
        [0., 0., 0.]])